# 3D Surface detection

This notebook serves as a sandbox for the SimpleITK framework and will be used to derive the 3D surfaces from the single cell images.

---

## 0. Environmental setup

In [1]:
import matplotlib.pyplot as plt
import SimpleITK as sitk
import tifffile
import sys
import numpy as np
from skimage import color, filters, segmentation, exposure, morphology
import scipy.ndimage as ndi
import copy

sys.path.append('..')

from src.utils.io import get_file_list

!export SITK_SHOW_COMMAND='/home/daniel/Applications/Fiji.app/ImageJ-linux64'

In [2]:
def show_plane(ax, plane, cmap="gray", title=None):
    ax.imshow(plane, cmap=cmap)
    ax.axis("off")

    if title:
        ax.set_title(title)

        
def explore_slices(data, cmap="gray"):
    from ipywidgets import interact
    N = len(data)

    @interact(plane=(0, N - 1))
    def display_slice(plane=34):
        fig, ax = plt.subplots(figsize=(20, 5))

        show_plane(ax, data[plane], title="Plane {}".format(plane), cmap=cmap)

        plt.show()

    return display_slice

def explore_slices_2_samples(data, cmap="gray"):
    from ipywidgets import interact
    N = len(data[0])

    @interact(plane=(0, N - 1))
    def display_slice(plane=34):
        fig, ax = plt.subplots(figsize=(20, 5), nrows=1, ncols=2)

        show_plane(ax[0], data[0][plane], title="Plane {}".format(plane), cmap=cmap)
        show_plane(ax[1], data[1][plane], title='Plane {}'.format(plane), cmap=cmap)

        plt.show()

    return display_slice


---

## 1. Read in data

In [3]:
root_dir = '../data/tcell_project/filtered/selected_slices_ar_08_area25_50_16bit'
root_dir = '/home/daniel/PycharmProjects/nuclear_segmentation/data/tcell_project/filtered/fish_ar_08_area_400sqmicron/filtered/'
file_list = get_file_list(root_dir)

cell_ids = list(range(20))
cells = []

for i in range(len(cell_ids)):
    cells.append(np.squeeze(tifffile.imread(file_list[cell_ids[i]])))

In [25]:
print(file_list[:20])

['/home/daniel/PycharmProjects/nuclear_segmentation/data/tcell_project/filtered/fish_ar_08_area_400sqmicron/filtered/t_cell_chr1_chr3_activated_2_s0_n0.tiff', '/home/daniel/PycharmProjects/nuclear_segmentation/data/tcell_project/filtered/fish_ar_08_area_400sqmicron/filtered/t_cell_chr1_chr3_activated_3_s0_n0.tiff', '/home/daniel/PycharmProjects/nuclear_segmentation/data/tcell_project/filtered/fish_ar_08_area_400sqmicron/filtered/t_cell_chr1_chr3_activated_3_s0_n1.tiff', '/home/daniel/PycharmProjects/nuclear_segmentation/data/tcell_project/filtered/fish_ar_08_area_400sqmicron/filtered/t_cell_chr1_chr3_activated_5_s0_n0.tiff', '/home/daniel/PycharmProjects/nuclear_segmentation/data/tcell_project/filtered/fish_ar_08_area_400sqmicron/filtered/t_cell_chr1_chr3_activated_5_s0_n1.tiff', '/home/daniel/PycharmProjects/nuclear_segmentation/data/tcell_project/filtered/fish_ar_08_area_400sqmicron/filtered/t_cell_chr1_chr3_activated_6_s0_n0.tiff', '/home/daniel/PycharmProjects/nuclear_segmentation/

In [26]:
dapi_imgs = []
chr1_imgs = []
chr3_imgs = []
for i in range(len(cells)):
    dapi_imgs.append(cells[i][:,0,:,:])
    chr1_imgs.append(cells[i][:,1,:,:])
    chr3_imgs.append(cells[i][:,2,:,:])

In [5]:
#dapi_img = cells[0]
cell_img = sitk.GetImageFromArray(dapi_img.astype(np.float32))

In [6]:
explore_slices(sitk.GetArrayFromImage(cell_img))

interactive(children=(IntSlider(value=23, description='plane', max=23), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices.<locals>.display_slice(plane=34)>

---

## 2. Canny Edge detection in 3D

In [22]:
edges = sitk.CannyEdgeDetection(cell_img, lowerThreshold=0.0, 
                                upperThreshold=20.0, variance = (2.0,2.0,2.0))

In [23]:
explore_slices(sitk.GetArrayFromImage(edges))

interactive(children=(IntSlider(value=23, description='plane', max=23), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices.<locals>.display_slice(plane=34)>

---

## 3. Chromosome masks

In the following we will aim to derive the chromosome masks.

Let us first have a blink at the raw data.

In [9]:
explore_slices(chr_img_3)

interactive(children=(IntSlider(value=23, description='plane', max=23), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices.<locals>.display_slice(plane=34)>

In [10]:
def get_structure_mask(dapi_img, mask=None):
    filtered = exposure.adjust_gamma(filters.median(dapi_img),0.5)
    #cell_mask = segmentation.morphological_chan_vese(filtered, 500)
    t = filters.threshold_otsu(filtered)
    binary = filtered > t
    cell_mask = []
    for i in range(len(filtered)):
        filled = ndi.binary_fill_holes(binary[i])
        filled = morphology.remove_small_objects(filled)
        cell_mask.append(filled)
    cell_mask = np.array(cell_mask)
    return cell_mask

In [16]:
def get_masked_chromosome_image(chr_img, nuclei_mask):
    chr_mask = np.zeros_like(chr_img)
    chr_mask[nuclei_mask] = chr_img[nuclei_mask]
    return chr_mask

In [17]:
nuclei_mask = get_structure_mask(dapi_img)
masked_chr_img_1 = get_masked_chromosome_image(chr_img_1, nuclei_mask)

In [21]:
explore_slices_2_samples([dapi_img, masked_chr_img_1])

interactive(children=(IntSlider(value=23, description='plane', max=23), Output()), _dom_classes=('widget-inter…

<function __main__.explore_slices_2_samples.<locals>.display_slice(plane=34)>

In [ ]:
surface_mask = sitk.GetArrayFromImage(edges)
for i in range(len(surface_mask))

---
